In [1]:
import chess

from chess_app.environment import ChessInterface
from re_algoritms.agents import RandomAgent, QAgent
from chess_app.dataset.loader import loaderChessPos
import chess.svg
from tqdm import tqdm


In [2]:
from sklearn.feature_extraction.text import HashingVectorizer
# list of text documents
text = ["The quick brown fox jumped over the lazy dog."]
# create the transform
vectorizer = HashingVectorizer(n_features=20)
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

(1, 20)
[[ 0.          0.          0.          0.          0.          0.33333333
   0.         -0.33333333  0.33333333  0.          0.          0.33333333
   0.          0.          0.         -0.33333333  0.          0.
  -0.66666667  0.        ]]


In [27]:
import numpy as np

fen_string = 'rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2'
fen_string_2 = 'rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/rNBQKBNR w KQkq - 0 2'
vectorizer = HashingVectorizer(n_features=32, lowercase=False, analyzer='char')
vec1 = vectorizer.transform([fen_string]).toarray()
vec2 = vectorizer.transform([fen_string_2]).toarray()

assert np.array_equal(vec1, vec2)

AssertionError: 

In [89]:
move1 = 'a2a5'
move2 = 'a2a9'
vectorizer = HashingVectorizer(n_features=5, lowercase=False, analyzer='word')
vec1 = vectorizer.transform([move1]).toarray()
vec2 = vectorizer.transform([move2]).toarray()

print(vec1, vec2)
assert np.array_equal(vec1, vec2)

[[0. 0. 0. 1. 0.]] [[0. 0. 1. 0. 0.]]


AssertionError: 

In [102]:
def move_to_number(move: str) -> float:
    letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
    nums = [str(i + 1) for i in range(len(letters))]
    letters_to_num = dict(zip(letters, nums))

    move = list(move)
    move[0] = letters_to_num[move[0]]
    move[2] = letters_to_num[move[2]]
    move = ''.join(move)
    print(move)

    move = int(move)

    return (move - 1111) / (8888 - 1111)

move_to_number('b1c3')

2133


0.13141314131413143

In [2]:

def is_checkmate(board, state):
    board.set_fen(state)
    return board.is_checkmate()


In [3]:
board = chess.Board()


e = 0.1
lr = 0.3
gamma = 0.3
n_epochs = 100
player = QAgent(lr, gamma)
loader = loaderChessPos(r'./chess_app/dataset')

In [4]:
bc_win = 0
wh_win = 0
stockfish = ChessInterface(verbose=True, engine_path=r"./chess_app/src/stockfish_15_x64_avx2.exe")
for epoch in range(n_epochs):
    for cnt_to_finish in list(loader.games.keys()):
        for state in tqdm(loader.games[cnt_to_finish]):
            stockfish.stockfish.set_fen_position(state)
            actions = list()

            move_counter = 1
            reward = 0
            black_win = False
            white_win = False
            new_state = state
            b_state = None
            while not is_checkmate(board, new_state):
                stockfish.stockfish.set_fen_position(new_state)
                player_move = player.return_move(new_state, stockfish, e)
                #print(f'Player move:{player_move}')
                stockfish.player_move(player_move)
                #new_state = stockfish.stockfish.get_fen_position()
            # actions.append(new_state) # ?
                move_counter += 1
                if move_counter > 5:
                    black_win = True
                    bc_win += 1
                    break
                actions.append(new_state)

                if is_checkmate(board, new_state):
                    wh_win += 1
                    white_win = True
                else:
                    machine_move = stockfish.env_move()
                    new_state = stockfish.get_board_fen()
                    #print(f'Machine move:{machine_move}')
                if is_checkmate(board, new_state) and not white_win:
                    bc_win += 1
                    black_win = True
                print(actions)

                    

                #print('---')
                # break

            if black_win is True:
                reward = -1
            else:
                reward = 1
            print(player.q_table[actions[-2]])
            player.calculate_qvalue(actions, reward=reward)
            # break
        print(f'Win: {wh_win}\t Lose: {bc_win}')


  0%|          | 0/1217 [00:00<?, ?it/s]


KeyError: 'k1K5/8/2R5/8/8/8/8/8 w - - 0 1'

In [ ]:
player.q_table

In [ ]:
board.set_fen(state)

In [ ]:
board

In [ ]:
state = stockfish.reset()
board.set_fen(state)
board.push_san('e2e4')
board

In [ ]:
brd = chess.Board()
brd.push_san('e2e4')
brd.push_san('e7e6')
brd

In [ ]:
brd.fen()

In [ ]:
player.q_table